In [1]:
import pandas as pd
import numpy as np
import math
import scipy.linalg 
from scipy.linalg import block_diag
from scipy.stats import gamma 
from scipy.stats import norm
from scipy.stats import invgauss
from scipy.stats import invgamma
import scipy as sp
from scipy.sparse import coo_matrix
import time
import warnings
warnings.filterwarnings("ignore")
from numpy import asarray
from numpy import savetxt

In [2]:

## define random draws from a gen
def psi(x,alpha,lam):
    return -alpha*(scipy.cosh(x)-1)-lam*(scipy.exp(x) - x - 1)

def dpsi(x,alpha,lam):
    return -alpha*scipy.sinh(x) - lam*(scipy.exp(x) - 1)

def g(x,sd,td,f1,f2):
    a = 0
    b = 0
    c = 0
    if x >= -sd and x <= td:
        a=1
    elif x > td:
        b = f1
    elif x < -sd:
        c = f2
    return a + b + c    

def GIGrnd(p,a,b,sam):
    lam = p
    omega = math.sqrt(a*b)
    swap = 0
    if lam < 0:
        lam = lam*-1
        swap = 1
    alpha = scipy.sqrt(omega**2 + lam**2) - lam
    # Find t
    x = -psi(1,alpha,lam)
    if x >= .5 and x <= 2:
        t=1
    elif x > 2:
        t = scipy.sqrt(2/(alpha + lam))
    elif x < 0.5:
        t = scipy.log(4/(alpha + 2*lam))
    # Find s
    x = -psi(1,alpha,lam)
    if x >= .5 and x <= 2:
        s=1
    elif x > 2:
        s = scipy.sqrt(4/(alpha*scipy.cosh(1) + lam))
    elif x < 0.5:
        s = np.min(np.array([1/lam,scipy.log(1 + 1/alpha + scipy.sqrt(1/alpha**2 + 2/alpha))]).reshape(1,2))
    # Generation
    eta = -psi(t,alpha,lam)
    zeta = -dpsi(t,alpha,lam)
    theta = -psi(-s,alpha,lam)
    xi = dpsi(-s,alpha,lam)
    p = 1/xi
    r = 1/zeta
    td = t - r*eta
    sd = s - p*theta
    q = td + sd
    bigX = np.zeros((sam,1))
    for i in range (sam):
        dd = 1
        while dd > 0:
            U = np.random.rand(1)
            V = np.random.rand(1)
            W = np.random.rand(1)
            if U < (q/(p+q+r)):
                bigX[i,:] = -sd + q*V
            elif U < ((q+r)/(p+q+r)):
                bigX[i,:] = td - r*scipy.log(V)
            else:
                bigX[i,:] = -sd + p*scipy.log(V)
        
            f1 = scipy.exp(-eta - zeta*(bigX[i,:] -t))
            f2 = scipy.exp(-theta + xi*(bigX[i,:] + s))
            
            if W*g(bigX[i,:],sd,td,f1,f2) <= scipy.exp(psi(bigX[i,:],alpha,lam)):
                break
    bigX = scipy.exp(bigX)*(lam/omega + scipy.sqrt(1 + (lam/omega)**2))
    if swap == 1:
        bigX = 1/bigX
    return bigX/scipy.sqrt(a/b)

#Here are the number of draws and burnin
nsim = 100 # number of draws
burnin = 10 # number of burnin draws

In [3]:
ts = pd.read_csv("quarterly.csv",header=None) # Load quarterly dataset
dataq = ts.values

ts = pd.read_csv("annual.csv",header=None) # Load annual dataset
dataa = ts.values

In [4]:
tm = dataq[:,-1]
# expand the annual data to mimic quarterly data
ya = np.kron(dataa[:,1:],np.ones((4,1)))

In [6]:
# adjust data so that they have the same length
yq = dataq[:,1:-1]
Tnew = len(yq) - len(ya)
if Tnew > 0:
     ya = np.ma.row_stack((ya,np.ones((Tnew,1))*ya[-1,:]))  

In [17]:
# create a dataset with the lags 
ydata = np.column_stack((yq,ya))
p = 7 # no. of lags
quarters = tm[p:]
yact0 = ydata[0:p,:]
yact = ydata[p:,:]
y = ydata[p:,:]
ystore = y


In [18]:

nq = 5 # no. of quartely variables
na = 12 # no. of annual variables
n = nq + na
T = len(yact) # no. of periods
k = n+p*n**2 # no. of parameters
m = int((n*(n-1))*.5) # no. of covariance
Lid = np.nonzero(np.tril(np.ones((n,n)),-1))
L = np.eye(n)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0